In [27]:
import io
import torch
import torch.onnx
import onnx
from unet import UNet
import onnxruntime
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
# from utils.dataset import BasicDataset

In [28]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pthfile = 'ckpt.test.pth'
onnxpath = 'test_conv_pool.onnx'
print("torch.__version__:", torch.__version__)
print("onnx.__version__:", onnx.__version__)
print("onnxruntime.__version__:", onnxruntime.__version__)

torch.__version__: 1.4.0
onnx.__version__: 1.6.0
onnxruntime.__version__: 1.6.0


In [29]:
onnx_model = onnx.load(onnxpath)
check = onnx.checker.check_model(onnx_model)
print("check:", check)

check: None


In [30]:
def crop_map(h, crop_size):
    
#     bs = torch.tensor(1)
#     ch = torch.tensor(4)
#     H = torch.tensor(481)
#     W = torch.tensor(481)
    
    tmp_h_0 = torch.zeros(120, 481)
    tmp_h_1 = torch.ones(120, 481)
    tmp_v_0 = torch.zeros(721, 120)
    tmp_v_1 = torch.ones(721, 120)
    
    ############### first map ######################
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h0 = torch.cat((tmp_map[0][0], tmp_h_1), dim=0)
    tmp_h0 = torch.cat((tmp_h_1, tmp_h0), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v0 = torch.cat((tmp_h0, tmp_v_1), dim=1)
    tmp_v0 = torch.cat((tmp_v_1, tmp_v0), dim=1)
    
    ################ second map ####################
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h1 = torch.cat((tmp_map[0][1], tmp_h_0), dim=0)
    tmp_h1 = torch.cat((tmp_h_0, tmp_h1), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v1 = torch.cat((tmp_h1, tmp_v_0), dim=1)
    tmp_v1 = torch.cat((tmp_v_0, tmp_v1), dim=1)

    ################ Third map ####################
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h2 = torch.cat((tmp_map[0][2], tmp_h_0), dim=0)
    tmp_h2 = torch.cat((tmp_h_0, tmp_h2), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v2 = torch.cat((tmp_h2, tmp_v_0), dim=1)
    tmp_v2 = torch.cat((tmp_v_0, tmp_v2), dim=1)
    # print("tmp_v:", tmp_v.size())

    ################ Forth map ####################

    # tensor_index = torch.linspace(120., 601., 481)
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h3 = torch.cat((tmp_map[0][3], tmp_h_0), dim=0)
    tmp_h3 = torch.cat((tmp_h_0, tmp_h3), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v3 = torch.cat((tmp_h3, tmp_v_0), dim=1)
    tmp_v3 = torch.cat((tmp_v_0, tmp_v3), dim=1)
    
    ################################################
    map_tmp = torch.cat((torch.unsqueeze(tmp_v0, 0), torch.unsqueeze(tmp_v1, 0), 
                         torch.unsqueeze(tmp_v2, 0), torch.unsqueeze(tmp_v3, 0)), 0)
    map_tmp = torch.unsqueeze(map_tmp, 0)
    # print("after expanding map size:", map_tmp.size())

    ############ start crap map #####################
    x_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,0][0]
    y_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,1][0]
    
    
    # print(map_tmp)
    # print("x_pos:{}, y_pos:{}".format(x_pos,y_pos))
    # print(map_tmp.size())
    # print(torch.nonzero(map_tmp[0][3]==1))
    index_tensor = torch.linspace(-120, 119, 240)
    # index_tensor = torch.tensor([-120, -119, -118, -117, -116, -115, -114, -113, -112, -111, -110, -109, -108, -107, -106, -105, -104, -103, -102, -101, -100, -99, -98, -97, -96, -95, -94, -93, -92, -91, -90, -89, -88, -87, -86, -85, -84, -83, -82, -81, -80, -79, -78, -77, -76, -75, -74, -73, -72, -71, -70, -69, -68, -67, -66, -65, -64, -63, -62, -61, -60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48, -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35, -34, -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119])
    # index_tensor = index_tensor.long()
    index_tensor_x = index_tensor.long() + x_pos.long()
    index_tensor_y = index_tensor.long() + y_pos.long()
    # print("index_tensor_x", index_tensor_x)

    tmp_crap_map_0 = torch.index_select(map_tmp[0][0], 1, index_tensor_y)
    tmp_crap_map_0 = torch.index_select(tmp_crap_map_0, 0, index_tensor_x)
    
    tmp_crap_map_1 = torch.index_select(map_tmp[0][1], 1, index_tensor_y)
    tmp_crap_map_1 = torch.index_select(tmp_crap_map_1, 0, index_tensor_x)
    
    tmp_crap_map_2 = torch.index_select(map_tmp[0][2], 1, index_tensor_y)
    tmp_crap_map_2 = torch.index_select(tmp_crap_map_2, 0, index_tensor_x)
    
    tmp_crap_map_3 = torch.index_select(map_tmp[0][3], 1, index_tensor_y)
    tmp_crap_map_3 = torch.index_select(tmp_crap_map_3, 0, index_tensor_x)
    
    tmp = tmp_crap_map_3
    
    '''
    output = torch.cat((torch.unsqueeze(tmp_crap_map_0, 0), torch.unsqueeze(tmp_crap_map_1, 0),
                       torch.unsqueeze(tmp_crap_map_2, 0), torch.unsqueeze(tmp_crap_map_3, 0)), 0)
    output = torch.unsqueeze(output, 0)
    # print("ouput::::",output.size())
    '''
    # output = torch.randn(1, 4, crop_size, crop_size)
    '''
    output[0][0] = tmp_crap_map_0
    output[0][1] = tmp_crap_map_1
    output[0][2] = tmp_crap_map_2
    output[0][3] = tmp_crap_map_3
    '''
    
    tmp_0 = tmp_crap_map_0
    tmp_1 = tmp_crap_map_1
    tmp_2 = tmp_crap_map_2
    tmp_3 = tmp_crap_map_3
    tmp_0 = torch.reshape(tmp_0,[1,1,tmp_0.shape[0],tmp_0.shape[1]])
    tmp_1 = torch.reshape(tmp_1,[1,1,tmp_1.shape[0],tmp_1.shape[1]])
    tmp_2 = torch.reshape(tmp_2,[1,1,tmp_2.shape[0],tmp_2.shape[1]])
    tmp_3 = torch.reshape(tmp_3,[1,1,tmp_3.shape[0],tmp_3.shape[1]])
    
    tmp = torch.cat((tmp_0,tmp_1,tmp_2,tmp_3),1)
#     tmp_0 = torch.unsqueeze(tmp_crap_map_0, 0)
#     tmp_1 = torch.unsqueeze(tmp_crap_map_1, 0)
#     tmp_2 = torch.unsqueeze(tmp_crap_map_2, 0)
#     tmp_3 = torch.unsqueeze(tmp_crap_map_3, 0)
    
#     tmp = torch.cat((tmp_0, tmp_1), 0)
#     tmp = torch.cat((tmp, tmp_2), 0)
#     tmp = torch.cat((tmp, tmp_3), 0)
#     tmp = torch.unsqueeze(tmp, 0)
    # print("tmp size:", tmp.size())
    
    # tmp = torch.index_select(tmp, 0, torch.tensor([0]))
    # print("tmp size:", tmp.size())
    # output = tmp
    # print("output:", tmp.size())
    
    
    return tmp

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self, G):
        super().__init__()
        self.G = torch.tensor(G)
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            Flatten(),  # (G*G, )
            nn.Sigmoid(),  # added for non-negative
        )
        
        
    def _get_h12(self, inputs): # inputs needs to be a tensor, i.e., original inputs["map_at_t"], (bs, 4, M, M), channel 3 means one-hot pose, channel 0~1 means global map
        # x = inputs["pose_in_map_at_t"]  # (bs,2)
        # map_at_t (4, m, m)
        # x = torch.nonzero(inputs[0][3]==1)
        h = inputs
        
        h_1 = crop_map(h, self.G)
        # print(torch.nonzero(h_1[0][3]==1))
        h_2 = F.max_pool2d(h, (2, 2))
# adaptive_max_pool2d
        #print("h111:", h_1.size())
        #print("h22:", h_2.size())
    
        h_12 = torch.cat((h_1, h_2), dim=1)

        return h_12

    def forward(self, inputs):
        x1 = self._get_h12(inputs)
        x2 = self.actor(x1)
        return x2

In [34]:
inputs = torch.randn(1, 4, 481, 481)

map3 = torch.zeros(1, 481, 481)
map3[0][20][20] = 1.
print(map3)
print(torch.nonzero(map3==1))
inputs[0][3] = map3
print(torch.nonzero(inputs[0][3]==1))

rl_net = Global_Actor(240)
rl_net.actor.load_state_dict(torch.load(pthfile, map_location='cpu'),strict=False)
rl_net.eval()

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
tensor([[ 0, 20, 20]])
tensor([[20, 20]])


Global_Actor(
  (actor): Sequential(
    (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(4, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(4, 2, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (10): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(2, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (13): Flatten()
    (14): Sigmoid()
  )
)

In [35]:
x = inputs
print(x.size())
torch_out = rl_net(x)
print("pth model output:", torch_out)
print(torch_out.shape)

ort_session = onnxruntime.InferenceSession(onnxpath)

# compute ONNX Runtime output prediction
ort_outs = ort_session.run(None, {ort_session.get_inputs()[0].name: x.cpu().numpy().astype(np.float32)})

# compare ONNX Runtime and PyTorch results

print("onnx model output:", ort_outs)
print('tor_out: ', torch_out.shape)

#np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
#print("Exported model has been tested with ONNXRuntime, and the result looks good!")

torch.Size([1, 4, 481, 481])
pth model output: tensor([[0.4694, 0.4706, 0.4692,  ..., 0.4675, 0.4727, 0.4738]],
       grad_fn=<SigmoidBackward>)
torch.Size([1, 57600])
onnx model output: [array([[0.47428057, 0.4646493 , 0.46358562, ..., 0.4679515 , 0.468348  ,
        0.47043318]], dtype=float32)]
tor_out:  torch.Size([1, 57600])


2021-12-02 11:17:26.765408255 [W:onnxruntime:, graph.cc:1069 Graph] Initializer actor.0.bias appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2021-12-02 11:17:26.767253293 [W:onnxruntime:, graph.cc:1069 Graph] Initializer actor.0.weight appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2021-12-02 11:17:26.767263465 [W:onnxruntime:, graph.cc:1069 Graph] Initializer actor.1.bias appears in graph inpu

In [33]:
def crop_map(h, crop_size):
    
#     bs = torch.tensor(1)
#     ch = torch.tensor(4)
#     H = torch.tensor(481)
#     W = torch.tensor(481)
    
    tmp_h_0 = torch.zeros(120, 481)
    tmp_h_1 = torch.ones(120, 481)
    tmp_v_0 = torch.zeros(721, 120)
    tmp_v_1 = torch.ones(721, 120)
    
    ############### first map ######################
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h0 = torch.cat((tmp_map[0][0], tmp_h_1), dim=0)
    tmp_h0 = torch.cat((tmp_h_1, tmp_h0), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v0 = torch.cat((tmp_h0, tmp_v_1), dim=1)
    tmp_v0 = torch.cat((tmp_v_1, tmp_v0), dim=1)
    
    ################ second map ####################
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h1 = torch.cat((tmp_map[0][1], tmp_h_0), dim=0)
    tmp_h1 = torch.cat((tmp_h_0, tmp_h1), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v1 = torch.cat((tmp_h1, tmp_v_0), dim=1)
    tmp_v1 = torch.cat((tmp_v_0, tmp_v1), dim=1)

    ################ Third map ####################
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h2 = torch.cat((tmp_map[0][2], tmp_h_0), dim=0)
    tmp_h2 = torch.cat((tmp_h_0, tmp_h2), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v2 = torch.cat((tmp_h2, tmp_v_0), dim=1)
    tmp_v2 = torch.cat((tmp_v_0, tmp_v2), dim=1)
    # print("tmp_v:", tmp_v.size())

    ################ Forth map ####################

    # tensor_index = torch.linspace(120., 601., 481)
    tmp_map = h
    # print("h size::", h.size())
    
    tmp_h3 = torch.cat((tmp_map[0][3], tmp_h_0), dim=0)
    tmp_h3 = torch.cat((tmp_h_0, tmp_h3), dim=0)
    # print("tmp_h:", tmp_h.size())

    tmp_v3 = torch.cat((tmp_h3, tmp_v_0), dim=1)
    tmp_v3 = torch.cat((tmp_v_0, tmp_v3), dim=1)
    
    ################################################
    map_tmp = torch.cat((torch.unsqueeze(tmp_v0, 0), torch.unsqueeze(tmp_v1, 0), 
                         torch.unsqueeze(tmp_v2, 0), torch.unsqueeze(tmp_v3, 0)), 0)
    map_tmp = torch.unsqueeze(map_tmp, 0)
    # print("after expanding map size:", map_tmp.size())

    ############ start crap map #####################
    x_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,0][0]
    y_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,1][0]
    
    
    # print(map_tmp)
    # print("x_pos:{}, y_pos:{}".format(x_pos,y_pos))
    # print(map_tmp.size())
    # print(torch.nonzero(map_tmp[0][3]==1))
    index_tensor = torch.linspace(-120, 119, 240)
    # index_tensor = torch.tensor([-120, -119, -118, -117, -116, -115, -114, -113, -112, -111, -110, -109, -108, -107, -106, -105, -104, -103, -102, -101, -100, -99, -98, -97, -96, -95, -94, -93, -92, -91, -90, -89, -88, -87, -86, -85, -84, -83, -82, -81, -80, -79, -78, -77, -76, -75, -74, -73, -72, -71, -70, -69, -68, -67, -66, -65, -64, -63, -62, -61, -60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48, -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35, -34, -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119])
    # index_tensor = index_tensor.long()
    index_tensor_x = index_tensor.long() + x_pos.long()
    index_tensor_y = index_tensor.long() + y_pos.long()
    # print("index_tensor_x", index_tensor_x)

    tmp_crap_map_0 = torch.index_select(map_tmp[0][0], 1, index_tensor_y)
    tmp_crap_map_0 = torch.index_select(tmp_crap_map_0, 0, index_tensor_x)
    
    tmp_crap_map_1 = torch.index_select(map_tmp[0][1], 1, index_tensor_y)
    tmp_crap_map_1 = torch.index_select(tmp_crap_map_1, 0, index_tensor_x)
    
    tmp_crap_map_2 = torch.index_select(map_tmp[0][2], 1, index_tensor_y)
    tmp_crap_map_2 = torch.index_select(tmp_crap_map_2, 0, index_tensor_x)
    
    tmp_crap_map_3 = torch.index_select(map_tmp[0][3], 1, index_tensor_y)
    tmp_crap_map_3 = torch.index_select(tmp_crap_map_3, 0, index_tensor_x)
    
    tmp = tmp_crap_map_3
    
    '''
    output = torch.cat((torch.unsqueeze(tmp_crap_map_0, 0), torch.unsqueeze(tmp_crap_map_1, 0),
                       torch.unsqueeze(tmp_crap_map_2, 0), torch.unsqueeze(tmp_crap_map_3, 0)), 0)
    output = torch.unsqueeze(output, 0)
    # print("ouput::::",output.size())
    '''
    # output = torch.randn(1, 4, crop_size, crop_size)
    '''
    output[0][0] = tmp_crap_map_0
    output[0][1] = tmp_crap_map_1
    output[0][2] = tmp_crap_map_2
    output[0][3] = tmp_crap_map_3
    '''
    
    tmp_0 = tmp_crap_map_0
    tmp_1 = tmp_crap_map_1
    tmp_2 = tmp_crap_map_2
    tmp_3 = tmp_crap_map_3
    tmp_0 = torch.reshape(tmp_0,[1,1,tmp_0.shape[0],tmp_0.shape[1]])
    tmp_1 = torch.reshape(tmp_1,[1,1,tmp_1.shape[0],tmp_1.shape[1]])
    tmp_2 = torch.reshape(tmp_2,[1,1,tmp_2.shape[0],tmp_2.shape[1]])
    tmp_3 = torch.reshape(tmp_3,[1,1,tmp_3.shape[0],tmp_3.shape[1]])
    
    tmp = torch.cat((tmp_0,tmp_1,tmp_2,tmp_3),1)
#     tmp_0 = torch.unsqueeze(tmp_crap_map_0, 0)
#     tmp_1 = torch.unsqueeze(tmp_crap_map_1, 0)
#     tmp_2 = torch.unsqueeze(tmp_crap_map_2, 0)
#     tmp_3 = torch.unsqueeze(tmp_crap_map_3, 0)
    
#     tmp = torch.cat((tmp_0, tmp_1), 0)
#     tmp = torch.cat((tmp, tmp_2), 0)
#     tmp = torch.cat((tmp, tmp_3), 0)
#     tmp = torch.unsqueeze(tmp, 0)
    # print("tmp size:", tmp.size())
    
    # tmp = torch.index_select(tmp, 0, torch.tensor([0]))
    # print("tmp size:", tmp.size())
    # output = tmp
    # print("output:", tmp.size())
    
    
    return tmp

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self, G):
        super().__init__()
        # self.G = torch.tensor(G)
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            Flatten(),  # (G*G, )
            nn.Sigmoid(),  # added for non-negative
        )
        
        
    def _get_h12(self, inputs): # inputs needs to be a tensor, i.e., original inputs["map_at_t"], (bs, 4, M, M), channel 3 means one-hot pose, channel 0~1 means global map
        # x = inputs["pose_in_map_at_t"]  # (bs,2)
        # map_at_t (4, m, m)
        # x = torch.nonzero(inputs[0][3]==1)
        h = inputs
        
        h_1 = crop_map(h, 240)
        # print(torch.nonzero(h_1[0][3]==1))
        h_2 = F.max_pool2d(h, (2, 2))
# adaptive_max_pool2d
        #print("h111:", h_1.size())
        #print("h22:", h_2.size())
    
        h_12 = torch.cat((h_1, h_2), dim=1)

        return h_12

    def forward(self, inputs):
        x1 = self._get_h12(inputs)
        x2 = self.actor(x1)
        return x2

In [11]:
# print(torch.linspace(-120, 119, 240))